In [ ]:
import cv2
import face_recognition
import numpy as np

def faces_comparison_using_webcam():
    # Inicializa la captura de video desde la webcam
    video_capture = cv2.VideoCapture(0, cv2.CAP_DSHOW)

    # Verifica si la cámara pudo abrirse correctamente
    if not video_capture.isOpened():
        print("Error: No se pudo abrir la cámara.")
        return

    while True:
        # Lee el siguiente fotograma de la cámara
        ret, frame = video_capture.read()
        
        if not ret:
            print("Error: No se pudo leer un fotograma de la cámara.")
            break

        # Convertir el fotograma de BGR (OpenCV) a RGB (face_recognition)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detecta ubicaciones de caras en el frame
        face_locations = face_recognition.face_locations(rgb_frame)
        # Obtener los encodings de las caras detectadas
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Dibujar un recuadro alrededor de cada cara detectada
        for (top, right, bottom, left) in face_locations:
            # Asegurarse de que las coordenadas estén dentro del frame
            height, width = frame.shape[:2]
            left, right, top, bottom = [max(0, min(val, dim)) for val, dim in zip((left, right, top, bottom), (width, width, height, height))]

            # Dibujar el recuadro alrededor de la cara detectada
            color = (0, 255, 0)  # Verde
            thickness = 2  # Grosor del rectángulo
            cv2.rectangle(frame, (left, top), (right, bottom), color, thickness)

        # Comparar todas las caras detectadas entre sí
        if len(face_encodings) >= 2:
            font = cv2.FONT_HERSHEY_DUPLEX
            for i in range(len(face_encodings)):
                for j in range(i+1, len(face_encodings)):
                    # Calcular la distancia de similitud entre las caras
                    distance = face_recognition.face_distance([face_encodings[i]], face_encodings[j])[0]
                    
                    # Convertir la distancia en porcentaje de similitud
                    similarity_percentage = (1 - distance) * 100
                    print(f"Similitud entre cara {i+1} y cara {j+1}: {similarity_percentage:.2f}%")

                    # Mostrar la similitud en el frame, en algún punto intermedio entre las dos caras
                    top1, right1, bottom1, left1 = face_locations[i]
                    top2, right2, bottom2, left2 = face_locations[j]
                    
                    # Calcular los puntos centrales de las dos caras
                    center_x1, center_y1 = (left1 + right1) // 2, (top1 + bottom1) // 2
                    center_x2, center_y2 = (left2 + right2) // 2, (top2 + bottom2) // 2
                    
                    # Dibujar una línea recta entre los puntos centrales de las dos caras
                    line_color = (255, 0, 0)  # Azul
                    line_thickness = 2
                    cv2.line(frame, (center_x1, center_y1), (center_x2, center_y2), line_color, line_thickness)

                    # Calcular el punto medio de la línea
                    midpoint_x = (center_x1 + center_x2) // 2
                    midpoint_y = (center_y1 + center_y2) // 2
                    
                    # Dibujar el porcentaje de similitud en la posición calculada
                    cv2.putText(frame, f"{similarity_percentage:.2f}%", (midpoint_x, midpoint_y), font, 0.7, (255, 255, 255), 2)
        else:
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, "No se han detectado suficientes caras", (50, 50), font, 1.0, (0, 0, 255), 2)

        # Mostrar el frame procesado en tiempo real
        cv2.imshow('Video', frame)

        # Salir con la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Liberar la cámara y cerrar ventanas
    video_capture.release()
    cv2.destroyAllWindows()

# Ejecutar el reconocimiento facial desde la webcam
faces_comparison_using_webcam()
